# Khoa học dữ liệu trên đám mây: Cách sử dụng "Azure ML SDK"

## Giới thiệu

Trong notebook này, chúng ta sẽ học cách sử dụng Azure ML SDK để huấn luyện, triển khai và sử dụng một mô hình thông qua Azure ML.

Điều kiện tiên quyết:
1. Bạn đã tạo một workspace Azure ML.
2. Bạn đã tải [dữ liệu Suy tim](https://www.kaggle.com/andrewmvd/heart-failure-clinical-data) vào Azure ML.
3. Bạn đã tải notebook này lên Azure ML Studio.

Các bước tiếp theo:

1. Tạo một Experiment trong Workspace hiện có.
2. Tạo một Compute cluster.
3. Tải dữ liệu.
4. Cấu hình AutoML bằng AutoMLConfig.
5. Chạy thí nghiệm AutoML.
6. Khám phá kết quả và lấy mô hình tốt nhất.
7. Đăng ký mô hình tốt nhất.
8. Triển khai mô hình tốt nhất.
9. Sử dụng endpoint.

## Các import cụ thể của Azure Machine Learning SDK


In [ ]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import AmlCompute
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice

## Khởi Tạo Không Gian Làm Việc
Khởi tạo một đối tượng không gian làm việc từ cấu hình đã được lưu trữ. Đảm bảo rằng tệp cấu hình có mặt tại .\config.json


In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

## Tạo một thí nghiệm Azure ML

Hãy tạo một thí nghiệm có tên 'aml-experiment' trong không gian làm việc mà chúng ta vừa khởi tạo.


In [ ]:
experiment_name = 'aml-experiment'
experiment = Experiment(ws, experiment_name)
experiment

## Tạo Cụm Tính Toán
Bạn sẽ cần tạo một [mục tiêu tính toán](https://docs.microsoft.com/azure/machine-learning/concept-azure-machine-learning-architecture#compute-target) cho lần chạy AutoML của mình.


In [ ]:
aml_name = "heart-f-cluster"
try:
    aml_compute = AmlCompute(ws, aml_name)
    print('Found existing AML compute context.')
except:
    print('Creating new AML compute context.')
    aml_config = AmlCompute.provisioning_configuration(vm_size = "Standard_D2_v2", min_nodes=1, max_nodes=3)
    aml_compute = AmlCompute.create(ws, name = aml_name, provisioning_configuration = aml_config)
    aml_compute.wait_for_completion(show_output = True)

cts = ws.compute_targets
compute_target = cts[aml_name]

## Dữ liệu
Hãy đảm bảo rằng bạn đã tải lên tập dữ liệu vào Azure ML và rằng khóa có cùng tên với tập dữ liệu.


In [ ]:
key = 'heart-failure-records'
dataset = ws.datasets[key]
df = dataset.to_pandas_dataframe()
df.describe()

## Cấu hình AutoML


In [ ]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 3,
    "primary_metric" : 'AUC_weighted'
}

automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

## Chạy AutoML


In [ ]:
remote_run = experiment.submit(automl_config)

In [ ]:
RunDetails(remote_run).show()

In [ ]:
best_run, fitted_model = remote_run.get_output()

In [ ]:
best_run.get_properties()

In [ ]:
model_name = best_run.properties['model_name']
script_file_name = 'inference/score.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')
description = "aml heart failure project sdk"
model = best_run.register_model(model_name = model_name,
                                description = description,
                                tags = None)

## Triển khai Mô hình Tốt nhất

Chạy đoạn mã sau để triển khai mô hình tốt nhất. Bạn có thể xem trạng thái triển khai trong cổng Azure ML. Bước này có thể mất vài phút.


In [ ]:
inference_config = InferenceConfig(entry_script=script_file_name, environment=best_run.get_environment())

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1,
                                               memory_gb = 1,
                                               tags = {'type': "automl-heart-failure-prediction"},
                                               description = 'Sample service for AutoML Heart Failure Prediction')

aci_service_name = 'automl-hf-sdk'
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

## Sử dụng Endpoint
Bạn có thể thêm các đầu vào vào mẫu đầu vào sau.


In [ ]:
data = {
    "data":
    [
        {
            'age': "60",
            'anaemia': "false",
            'creatinine_phosphokinase': "500",
            'diabetes': "false",
            'ejection_fraction': "38",
            'high_blood_pressure': "false",
            'platelets': "260000",
            'serum_creatinine': "1.40",
            'serum_sodium': "137",
            'sex': "false",
            'smoking': "false",
            'time': "130",
        },
    ],
}

test_sample = str.encode(json.dumps(data))

In [ ]:
response = aci_service.run(input_data=test_sample)
response


---

**Tuyên bố miễn trừ trách nhiệm**:  
Tài liệu này đã được dịch bằng dịch vụ dịch thuật AI [Co-op Translator](https://github.com/Azure/co-op-translator). Mặc dù chúng tôi cố gắng đảm bảo độ chính xác, xin lưu ý rằng các bản dịch tự động có thể chứa lỗi hoặc không chính xác. Tài liệu gốc bằng ngôn ngữ bản địa nên được coi là nguồn tham khảo chính thức. Đối với các thông tin quan trọng, nên sử dụng dịch vụ dịch thuật chuyên nghiệp từ con người. Chúng tôi không chịu trách nhiệm cho bất kỳ sự hiểu lầm hoặc diễn giải sai nào phát sinh từ việc sử dụng bản dịch này.
